In [1]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
import random
from scipy.ndimage import rotate, zoom
from sklearn.preprocessing import PowerTransformer
import pickle
from tqdm import tqdm
from multiprocessing import Pool
import gc

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from keras import models, layers, losses, optimizers, regularizers, Model
from keras.utils import plot_model

2023-11-10 18:52:57.403274: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 18:52:57.403415: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 18:52:57.403535: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
gc.collect()

0

In [3]:
NUM_CLASSES = 60

pickles = [pickle.load(open(f"pickles/{i}.pkl", "rb")) for i in range(3)]
targets = set([pickles[0][i][1] for i in range(len(pickles[0]))])

targets = list(targets)[:NUM_CLASSES]
targets = {target: [0 if i != j else 1 for i in range(NUM_CLASSES)] for j, target in enumerate(targets)}

pickles = [item for sublist in pickles for item in sublist if item[1] in targets]
pickles = sorted(pickles, key=lambda x: x[1])


def save_support():
    all_sum = np.add.reduce([item[0] for item in pickles]) / len(pickles)
    brain_index = all_sum != mode(all_sum.flat).mode

    with open("support.pkl", "wb") as f:
        pickle.dump(brain_index, f)

# save_support()

In [4]:
brain_index = pickle.load(open("support.pkl", "rb"))

pickles = [(item[0][brain_index], targets[item[1]]) for item in pickles]
trains, tests = [pickles[i] for i in range(len(pickles)) if i % 6 != 0], [pickles[i] for i in range(len(pickles)) if i % 6 == 0]

train_x = np.array([train[0] for train in trains])
train_y = np.array([train[1] for train in trains])

test_x = np.array([test[0] for test in tests])
test_y = np.array([test[1] for test in tests])

print(len(pickles), len(train_x), len(train_y), len(test_x), len(test_y))

1080 900 900 180 180


In [5]:
del pickles, trains, tests

In [6]:
class DenseMap(Model):
    def __init__(self):
        super().__init__()
        self.d1 = layers.Dense(128, activation="tanh", kernel_regularizer=regularizers.L1L2(0.1, 0.1), bias_regularizer=regularizers.L1L2(0.1, 0.1))
        self.d2 = layers.Dense(NUM_CLASSES, activation="softmax", kernel_regularizer=regularizers.L1L2(0.1, 0.1), bias_regularizer=regularizers.L1L2(0.1, 0.1))

        self.f = layers.Flatten()
        
    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)

        return x

In [7]:
model = DenseMap()

opt = optimizers.Adam(learning_rate=0.001)
loss = losses.CategoricalCrossentropy()

train_x, train_y = tf.convert_to_tensor(train_x), tf.convert_to_tensor(train_y)

model.compile(opt, loss)

for i in range(10000):
    with tf.GradientTape() as tape:
        pred_y = model(train_x)

        if i == 0:
            model.summary()

        batchloss = loss(train_y, pred_y)
        grad = tape.gradient(batchloss, model.trainable_variables)
        opt.apply_gradients(zip(grad, model.trainable_variables))

        print(i, float(batchloss))

    if i % 100 == 0 and i != 0:
        model.evaluate(test_x, test_y, batch_size=1)

Model: "dense_map"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  2927232   
                                                                 
 dense_1 (Dense)             multiple                  7740      
                                                                 
 flatten (Flatten)           multiple                  0 (unused)
                                                                 
Total params: 2934972 (11.20 MB)
Trainable params: 2934972 (11.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
0 4.242058753967285
1 4.396484375
2 3.456557035446167
3 2.89754319190979
4 2.5216424465179443
5 2.081024646759033
6 1.7387193441390991
7 1.4177863597869873
8 1.1399812698364258
9 0.9426484704017639
10 0.7749059200286865
11 0.6363500356674194
12 0.5275571942329407
13 0.429021090269